In [ ]:
from IPython.display import Audio, display
import librosa as li
import torch
import os
import random
torch.set_grad_enabled(False)

In [ ]:
number_files=5
sampling_rate=44100
cutting_length=10

In [ ]:
def load_audio_files(directory, number_files):
    all_files = os.listdir(directory)
    wav_files = [f for f in all_files if f.endswith('.wav')]
    random_files = random.sample(wav_files, number_files)
    
  
    audio_data = []
    for wav_file in random_files:
        file_path = os.path.join(directory, wav_file)
        y, sr = li.load(file_path, sr=sampling_rate)  # Load audio file
        audio_data.append(y) 

    return audio_data


In [ ]:

directory = 'data/musicnet/musicnet/train_data/'  

audio_data = load_audio_files(directory, number_files)

for audio in audio_data:
    display(Audio(audio, rate=sampling_rate))

In [ ]:
# url = 'https://play.forum.ircam.fr/rave-vst-api/get_model/musicnet'
#file_path = 'model.ts'
#download_file(url, file_path)

model = torch.jit.load("model.ts").eval()

In [ ]:
cutted_audio_data=[]

for audio in audio_data:
   cutted_audio_data.append(audio[:sampling_rate * cutting_length])

for cutted_audio in cutted_audio_data:
    display(Audio(cutted_audio, rate=sampling_rate))


In [ ]:
latent_data = []

for cutted_audio in cutted_audio_data:
    latent_data.append(model.encode(torch.from_numpy(cutted_audio).reshape(1,1,-1)))

merged_latent=sum(latent_data)/number_files
    
merged_audio = model.decode(merged_latent).numpy().reshape(-1)

#sf.write("output.wav", merged_audio, sampling_rate)
display(Audio(merged_audio, rate=sampling_rate))